### boston 집값 예측
target value: 실제 주택 가격

### NN regressor 모델 설계
- classification과 크게 코드차이가 없다.
- 차이:
1.   train셋과 test 셋 Scaler를 통해 transofrom을 해준다.
- 데이터의 분포가 다 다르기때문에 scaler를 맞춰주었다. preprocessing.StandardScaler() 를 사용.
2.   손실함수로 MSELoss를 사용한다.
3. 평가는 RMSE 




## baseline을 넘기 위해 내가 한 전략
- 1) 1layer 에서 3layer사용 -> 성능이 더 좋지 않음.
- 2) 3lyaer + dropout 적용-> **baseline을 넘겼다.**
- 3) 2번에서 13>256>1 을 13>512>1 로 변경했다. -> 성능이 조금 더 향상되었다. 

In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.1.1)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 64 Jun 15 09:42 kaggle.json


In [ ]:
!kaggle competitions download -c 2020aiboston

  0% 0.00/14.5k [00:00<?, ?B/s]
100% 14.5k/14.5k [00:00<00:00, 11.9MB/s]


In [ ]:
!unzip 2020aiboston.zip

Archive:  2020aiboston.zip
  inflating: boston_housing_test.csv  
  inflating: boston_housing_train.csv  
  inflating: sample_submission.csv   


In [ ]:
train_data=pd.read_csv('boston_housing_train.csv',header=None, usecols=range(1,15))
test_data=pd.read_csv('boston_housing_test.csv',header=None, usecols=range(1,14))

In [ ]:
import pandas as pd
import numpy as np

import torch
import torchvision.datasets as data
import torchvision.transforms as transforms
import random

from sklearn import preprocessing

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
# 학습 파라미터 설정
learning_rate = 0.1
training_epochs = 300
batch_size = 100

drop_prob = 0.3

Scaler = preprocessing.StandardScaler()  

# 데이터의 분포가 다 다르다. 이를 preprocessing.StandardScaler()를 이용해  Scaler를 맞춰주었다. 

In [ ]:
x_train_data=train_data.loc[:,0:13]
y_train_data=train_data.loc[:,14]

x_train_data=np.array(x_train_data)
y_train_data=np.array(y_train_data)
x_train_data = Scaler.fit_transform(x_train_data)

x_train_data=torch.FloatTensor(x_train_data)
y_train_data=torch.FloatTensor(y_train_data)      # y데이터가 FloatTensor 형이다.

In [ ]:
train_dataset = torch.utils.data.TensorDataset(x_train_data, y_train_data)

In [ ]:
# Tensor 데이터셋을 통해 DataLoader를 사용할 수 있다.

data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

![대체 텍스트](https://user-images.githubusercontent.com/11758940/83336289-229ec200-a2ed-11ea-9fb8-88a51198e475.png)

In [ ]:
linear1 = torch.nn.Linear(13,256,bias=True)
linear2 = torch.nn.Linear(256,256,bias=True)
linear3 = torch.nn.Linear(256,256,bias=True)
linear4 = torch.nn.Linear(256,1,bias=True)

relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

[메뉴얼] https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_

In [ ]:
# Random Init => Xavier Init
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)

Parameter containing:
tensor([[ 0.0356,  0.0085,  0.0999,  0.0127,  0.1249, -0.0273, -0.0803, -0.0915,
          0.0858, -0.0625,  0.1165,  0.1332, -0.0778, -0.1137,  0.0287,  0.0344,
          0.1482, -0.0813, -0.0298,  0.1211, -0.1243,  0.1513, -0.1211,  0.0975,
          0.0011, -0.0973, -0.0462, -0.0931,  0.0724,  0.0377,  0.1494, -0.0279,
          0.0648,  0.1255,  0.1171, -0.0611,  0.0891,  0.1256, -0.0317,  0.0219,
         -0.0679,  0.0643, -0.0786,  0.1386,  0.1097,  0.0322,  0.0389,  0.1511,
         -0.0558,  0.0662,  0.1030,  0.0803, -0.0410,  0.0009,  0.0152, -0.0615,
          0.1490,  0.1371, -0.0273, -0.1022, -0.0032, -0.0722, -0.1426,  0.0484,
          0.1214, -0.1152,  0.0584, -0.0672, -0.0806, -0.0023, -0.1467, -0.1266,
         -0.1166, -0.1104,  0.0042,  0.1296,  0.0402, -0.1188,  0.0973, -0.0215,
         -0.0950, -0.0459, -0.1372,  0.1183,  0.0629, -0.1350,  0.1478, -0.0270,
          0.1293, -0.0895,  0.0013, -0.0792,  0.0190,  0.0782,  0.1524, -0.1087,
      

In [ ]:
# ======================================
# relu는 맨 마지막 레이어에서 빼는 것이 좋다.
# ======================================

#model = torch.nn.Sequential(linear1).to(device)

model = torch.nn.Sequential(linear1, relu, dropout, 
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4).to(device)

[메뉴얼] https://pytorch.org/docs/stable/nn.html#crossentropyloss

## regressor는 손실함수로 MSELoss를 사용한다.

- 해당 손실함수로 배추값예측과 같은 선형회귀문제를 풀 수 있다.
- 분류문제에서는 softmax를 내부적으로 계산하는 CrossEntropyLoss()를 사용했다.

In [ ]:
# 손실함수와 최적화 함수
loss = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0001 cost = 1579533.625000000
Epoch: 0002 cost = 793.483093262
Epoch: 0003 cost = 489.312561035
Epoch: 0004 cost = 634.474365234
Epoch: 0005 cost = 644.295104980
Epoch: 0006 cost = 623.824157715
Epoch: 0007 cost = 641.734008789
Epoch: 0008 cost = 543.231750488
Epoch: 0009 cost = 500.645507812
Epoch: 0010 cost = 447.541015625
Epoch: 0011 cost = 468.923431396
Epoch: 0012 cost = 410.712707520
Epoch: 0013 cost = 369.593841553
Epoch: 0014 cost = 342.590057373
Epoch: 0015 cost = 302.263763428
Epoch: 0016 cost = 296.847106934
Epoch: 0017 cost = 254.333984375
Epoch: 0018 cost = 231.493316650
Epoch: 0019 cost = 212.317230225
Epoch: 0020 cost = 182.187927246
Epoch: 0021 cost = 191.683166504
Epoch: 0022 cost = 195.995635986
Epoch: 0023 cost = 188.741271973
Epoch: 0024 cost = 191.706817627
Epoch: 0025 cost = 202.914581299
Epoch: 0026 cost = 188.784255981
Epoch: 0027 cost = 182.536666870
Epoch: 0028 cost = 160.969024658
Epoch: 0029 cost = 191.067718506
Epoch: 0030 cost = 182.472671509
Epoch:



### test 셋도 Scaler를 통해 transofrom을 해준다.



1.   train 셋의 transfrom:   **fit을 사용**
- Scaler.fit_transform(x_train_data)
- fit -> train 데이터셋을 통해서 어떤 방식으로 scaler를 할지 계산을 구한다는 의미.

2. test셋의 transform: **그냥 transform만**
-  Scaler.transform(x_test_data) 
- fit_transform(train)을 통해 구해진 scaler 방식을 test셋에 적용하겠다는 의미. 

In [ ]:
# Test the model using test sets
with torch.no_grad():

  x_test_data=test_data.loc[:,:]
  x_test_data=np.array(x_test_data)
  x_test_data = Scaler.transform(x_test_data)         # test 셋도 Scaler를 통해 transofrom을 해준다.      
  x_test_data=torch.from_numpy(x_test_data).float().to(device)

  prediction = model(x_test_data)

# argmax를 하지 않는다. predication자체가 예측값이 되기 때문에.

In [ ]:
correct_prediction = prediction.cpu().numpy().reshape(-1,1)

In [ ]:
submit=pd.read_csv('sample_submission.csv')
submit

,id,Price
0,0.0,1.0
1,1.0,1.0
2,2.0,1.0
3,3.0,1.0
4,4.0,1.0
...,...,...
162,162.0,1.0
163,163.0,1.0
164,164.0,1.0
165,165.0,1.0


In [ ]:
for i in range(len(correct_prediction)):
  submit['Price'][i]=correct_prediction[i].item()

submit

,id,Price
0,0.0,25.778234
1,1.0,14.252911
2,2.0,15.069804
3,3.0,25.778234
4,4.0,15.069804
...,...,...
162,162.0,25.778234
163,163.0,15.069804
164,164.0,25.778234
165,165.0,25.778234


In [ ]:
submit.to_csv('baseline.csv',index=False,header=True)

# !kaggle competitions submit -c 2020aiboston -f baseline.csv -m "3layer dropout, 13>512>1"

### Evaluation: RMSE
- 해당 regression문제의 Evaluation은 RMSE 이다. (분류문제는 accuracy를 사용)
- 0에 가까운 값이 이상적인 값(답)이다.